In [39]:
import pandas as pd 
import pandas_profiling
from category_encoders.target_encoder import TargetEncoder as te
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor as rndf
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import r2_score, mean_absolute_error,mean_squared_error


In [40]:
def target_transform(x):
    trans = te()
    x['county_weight'] = trans.fit_transform(x.county.values,x.price).values/10e6
    return x.drop('county',axis=1)

In [41]:
def one_hot(x):
    county_dummies = pd.get_dummies(x.county)
    df1 = pd.concat([x, county_dummies], axis=1)
    return df1.drop('county',axis=1)

In [42]:
def split(x, test_size):
    inputs = x.drop(to_drop,axis=1)
    targets = x.price
    return train_test_split(inputs, targets, test_size=test_size, random_state=1)

In [43]:
def random_forest(x):  
    inputs = x.drop(['price'],axis=1)
    targets = x.price
    x_train, x_test, y_train, y_test = train_test_split(inputs, targets, test_size=0.2, random_state=1)
    rf = rndf()
    rf.fit(x_train, y_train)
    print('Random forest')
    print('train score: ',rf.score(x_train, y_train))
    print('test score: ',rf.score(x_test, y_test))
    y_pred = rf.predict(x_test)
    
    return rf


In [44]:
def test(f, to_drop):
    target_transformed = target_transform(f)
    one_hot_encoded = one_hot(f)
    
    print('Target transformed results')
    random_forest(target_transformed,to_drop)
    
    print('--------------------')
    print('One hot encoded results')
    
    random_forest(one_hot_encoded,to_drop) 

In [45]:
def outliers(f, z_score):
    df3 = f[(np.abs(stats.zscore(f[['area','price']])) < z_score).all(axis=1)]
    return df3

In [46]:
f = pd.read_csv('ready_for_testing.csv')


In [47]:
tt = target_transform(f)

In [48]:
out1 = outliers(tt,3)

In [49]:
random_forest(out)

Random forest
train score:  0.9731610459282147
test score:  0.8719440362004399


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [50]:
oh = one_hot(f)

In [51]:
out2 = outliers(oh, 3)

In [52]:
random_forest(out)

Random forest
train score:  0.9731890369406648
test score:  0.8683065099933912


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)